In [20]:
import pymongo
client = pymongo.MongoClient()
db = client['tripadvisor']
collection = db['attractionsBycountry']

In [21]:
from bson.son import SON
import numpy

In [100]:
# Check if pipeline aggregation function is correct and it is 
z= list(collection.find({'country':'Zimbabwe'}))
c= 0
tot = 0
for att in z:
    for a in att['attractions']:
        c+=1
        #tot += a['vgood']
        if 'excellent' in a:
            tot += a['excellent']
        #break
    

In [22]:
numpy.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [23]:
pipeline = [
    #{ "$project" : { "text" : "$attractions.sep_nov" }},
    {"$unwind" : "$attractions"},
    {"$group": {"_id": '$country',"average": {"$sum": '$attractions.average'},"poor": {"$sum": '$attractions.poor'},"terrible": {"$sum": '$attractions.terrible'},"vgood": {"$sum": '$attractions.vgood'},"excellent": {"$sum": '$attractions.excellent'},"business": {"$sum": '$attractions.business'},"solo": {"$sum": '$attractions.solo'},"friend": {"$sum": '$attractions.friend'},"family": {"$sum": '$attractions.family'}, "couple": {"$sum": '$attractions.couple'}, "mar_may": {"$sum": '$attractions.mar_may'},"jun_aug": {"$sum": '$attractions.jun_aug'},"sep_nov": {"$sum": '$attractions.sep_nov'}, "dec_feb": {"$sum": '$attractions.dec_feb'}}},
    {"$sort": SON([("count", -1), ("_id", -1)])}
]
agg_data = list(collection.aggregate(pipeline))

In [24]:
agg_countries= []
# create a list of dict storing the aggregate country data

for country in agg_data:
    try:
        data = {}
        data['country']= country['_id'] #dont aggregate the country
        #extract values and normalize the values x1 - u /sigma
        keys = [key for key in country.keys() if key !='_id']
        #seasons= [country[k] for k in keys if k=='dec_feb' or k=='mar_may' or k=='jun_aug' or k=='sep_nov']
        # Make a subset of the seasons
        seasons = { 'dec_feb','mar_may','jun_aug','sep_nov'}
        s_dict = { key:value for key,value in country.items() if key in seasons }

        # Make a subset of the traveller type
        trav_type = { 'couple','family','friend','business','solo'}
        t_dict = { key:value for key,value in country.items() if key in trav_type }

        # Make a subset of the overall rating
        rating = { 'excellent','vgood','average','terrible','poor'}
        r_dict = { key:value for key,value in country.items() if key in rating }

        #for category is season,calc mean metric/sum min-max
        def normalize(key_values):
            mean_metric = numpy.mean(key_values.values())
            std_metric = numpy.std(key_values.values())
            #print mean_metric,std_metric
            for k in key_values.keys():
                data[k]= round(((1.0 * country[k] - mean_metric)/(std_metric)),2)

        normalize(s_dict)
        normalize(t_dict)
        normalize(r_dict)
        agg_countries.append(data)
    except:
        print country
        break

In [25]:
print len(agg_countries)

234


In [26]:
agg_countries[0]

{u'average': -0.54,
 u'business': -1.4,
 'country': u'Zimbabwe',
 u'couple': 1.33,
 u'dec_feb': -0.72,
 u'excellent': 1.94,
 u'family': 0.13,
 u'friend': 0.76,
 u'jun_aug': 0.85,
 u'mar_may': -1.24,
 u'poor': -0.68,
 u'sep_nov': 1.11,
 u'solo': -0.83,
 u'terrible': -0.69,
 u'vgood': -0.03}

In [27]:
import pickle
with open('country_dt_means.p', 'rb') as fp:
    dt_data = pickle.load(fp)

In [28]:
# add dt field to agg_countries
for elem in agg_countries:
        try:
            dt = dt_data[elem['country']]
            elem['dt']=dt
        except:
            continue

In [29]:
agg_countries[230]

{u'average': -0.36,
 u'business': -0.71,
 'country': u'Algeria',
 u'couple': -0.84,
 u'dec_feb': -0.0,
 'dt': array([ 0.05772924,  0.        ,  0.01645799,  0.50342912,  0.05338187,
         0.11023541,  0.09463414,  0.16413223,  0.        ,  0.        ]),
 u'excellent': 1.42,
 u'family': 1.65,
 u'friend': 0.67,
 u'jun_aug': 0.47,
 u'mar_may': -1.59,
 u'poor': -0.95,
 u'sep_nov': 1.12,
 u'solo': -0.78,
 u'terrible': -1.04,
 u'vgood': 0.94}

In [30]:
from collections import Counter

In [31]:
country_at = {}
# get all the activity type for each country 
for country_data in list(collection.find()):
    agg_at = []
    for attr in country_data['attractions']:
        agg_at.append(attr['activity_types'])
    flat_at = [item for sublist in agg_at for item in sublist]
    mc = [ite for ite, it in Counter(flat_at).most_common(5)]
    country_at[country_data['country']]= mc
    

In [32]:
# add at to agg_countries
for elem in agg_countries:
        try:
            at = country_at[elem['country']]
            elem['at']=at
        except:
            continue

In [34]:
agg_countries[30]

{'at': [u'Mountains',
  u'Scenic Railroads',
  u'Bodies of Water',
  u'Neighborhoods',
  u'Ski & Snowboard Areas'],
 u'average': -0.6,
 u'business': -1.21,
 'country': u'Switzerland',
 u'couple': 1.65,
 u'dec_feb': -0.63,
 'dt': array([ 0.06209796,  0.00074326,  0.2581091 ,  0.10245742,  0.        ,
         0.15257498,  0.00078455,  0.04136237,  0.373284  ,  0.00858636]),
 u'excellent': 1.88,
 u'family': 0.4,
 u'friend': -0.03,
 u'jun_aug': 1.28,
 u'mar_may': -1.26,
 u'poor': -0.73,
 u'sep_nov': 0.61,
 u'solo': -0.81,
 u'terrible': -0.73,
 u'vgood': 0.17}

In [55]:
# extract features from agg_countries and store in numpy array
# feature order: excellent, vgood...
Y_labels = [] # store the country names
X = []
for cd in agg_countries:
    country_list = []
    skip = 0
    fields = ['excellent', 'vgood','average', 'poor','terrible','family', 'friend','couple','solo','business','dec_feb','mar_may','jun_aug','sep_nov','dt']
    for field in fields:
        # get the field value and store in X
        if field not in cd.keys():
            skip = 1
            break
        # flatten dt
        if field=='dt':
            for dt in cd['dt']:
                country_list.append(dt) 
        
        else:
            country_list.append(cd[field])
    if(not skip):
        Y_labels.append(cd['country'])
        X.append(numpy.asarray([country_list]))

In [256]:
X[0]

array([[ 1.94      , -0.03      , -0.54      , -0.68      , -0.69      ,
         0.13      ,  0.76      ,  1.33      , -0.83      , -1.4       ,
        -0.72      , -1.24      ,  0.85      ,  1.11      ,  0.13720219,
         0.0037902 ,  0.09591325,  0.15495553,  0.23691945,  0.11901002,
         0.        ,  0.07029533,  0.14941155,  0.03250248]])

In [56]:
X_train = numpy.asarray(X)

In [59]:
v= X_train.reshape(203,24)

In [310]:
v.shape

(203, 24)

In [37]:
from sklearn.cluster import KMeans

In [317]:
errors=[]
for n in range(5,22,3):
    km = KMeans(n_clusters= n, init='k-means++', max_iter=100, n_init=1,)
    km.fit(v)
    errors.append(calcError(v))

In [322]:
pyplot.plot(range(5,22,3), errors)
pyplot.show()

In [306]:
import numpy as np
from scipy import cluster
from matplotlib import pyplot

In [316]:
initial = [cluster.vq.kmeans(v,i) for i in range(10,20,4)]
pyplot.plot(range(10,20,4), [var for (cent,var) in initial])
pyplot.show()

In [319]:
km = KMeans(n_clusters= 17, init='k-means++', max_iter=100, n_init=1,)
km.fit(v)
print calcError(v)

30.8717248307


In [324]:
pred_cluster_ids_t[0]

14

In [348]:
hist(pred_cluster_ids_t)

In [349]:
data = [
    go.Histogram(
        x=pred_cluster_ids
    )
]
plot_url = py.plot(data, filename='10clusters')

/usr/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



In [325]:
pyplot.scatter(v[:,0], v[:,14], c=pred_cluster_ids_t)
pyplot.show()

In [320]:
pred_cluster_ids_t = km.predict(v)
Counter(pred_cluster_ids_t)

Counter({3: 35, 1: 24, 0: 14, 8: 13, 13: 13, 16: 13, 14: 11, 4: 10, 5: 10, 9: 10, 2: 9, 10: 9, 11: 8, 15: 8, 7: 7, 12: 6, 6: 3})

In [321]:
Counter(pred_cluster_ids)

Counter({8: 39, 3: 31, 6: 29, 2: 27, 1: 22, 4: 18, 0: 15, 5: 10, 9: 9, 7: 3})

In [264]:
import math

In [303]:
def error(point):
    
    center = km.cluster_centers_[km.predict(point)]
    center1= center[0]
    tot = 0
    for i in range(len(point)):
        d = point[0][i] - center1[i]
        tot += d**2
  
    return math.sqrt(tot)
    #return math.sqrt(sum([x**2 for x in (point - center1)]))

In [305]:
def calcError(v):
    WSSSE=0
    for v1 in v:
        WSSSE+=error(v1.reshape(1, -1))

    return WSSSE
    #WSSSE = v.map(lambda point: error(point)).reduce(lambda x, y: x + y)
    #print("Within Set Sum of Squared Error = " + str(WSSSE))


In [72]:
pred_cluster_ids = km.predict(v)
Counter(pred_cluster_ids)

In [75]:
from collections import Counter

In [76]:
Counter(pred_cluster_ids)

Counter({8: 39, 3: 31, 6: 29, 2: 27, 1: 22, 4: 18, 0: 15, 5: 10, 9: 9, 7: 3})

In [355]:
def findict(name):
    for d in agg_countries:
        if d['country']== name:
            return d
    return None

In [363]:
#clusters = {} 
nmatch_names= []
# get list of countries in each cluster
for cluster_id in range(17):
    
    indexes = numpy.where(pred_cluster_ids_t==cluster_id)
    for i in indexes[0]:
        cntry= Y_labels[i]
        cnt_dict= findict(cntry)
        if cnt_dict is not None:
            cnt_dict['cluster_id']= cluster_id
        else:
            nmatch_names.append(cntry)
    
        

In [492]:
pickle.dump(final_agg, open( "final_agg.p", "wb" ))
final_agg = pickle.load( open( "final_agg.p", "rb" ) )

In [493]:
import csv

In [495]:
keys = final_agg[0].keys()
with open('finalCOUNTRIES.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(final_agg)

In [484]:
# Filter out the agg_countries not in country_codes
for a in final_agg:
    if 'dt' in a:
        # get the value
        old=a['dt']
        a['dt']=old.tolist()
        

In [485]:
final_agg[0]

{'_id': ObjectId('56e21c201059e9f2c78e4ead'),
 'allowed_stay': u'3 months',
 'at': [u'Nature & Wildlife Areas',
  u'National Parks',
  u'Parks',
  u'Points of Interest & Landmarks',
  u'Historic Sites'],
 u'average': -0.54,
 u'business': -1.4,
 'cluster_id': 14,
 'country': u'Zimbabwe',
 'country_code': 'ZWE',
 u'couple': 1.33,
 u'dec_feb': -0.72,
 'dt': [0.13720219186556334,
  0.0037901973563552647,
  0.09591325374261457,
  0.1549555327661565,
  0.23691944949073118,
  0.11901001586072389,
  0.0,
  0.07029532807651526,
  0.14941155390152544,
  0.03250247693981457],
 u'excellent': 1.94,
 u'family': 0.13,
 u'friend': 0.76,
 u'jun_aug': 0.85,
 u'mar_may': -1.24,
 u'poor': -0.68,
 u'sep_nov': 1.11,
 u'solo': -0.83,
 u'terrible': -0.69,
 u'vgood': -0.03,
 'visa_req': u'Visa on arrival'}

In [481]:
final_agg=[]
c=0
for a in agg_countries:
    if 'country_code' in a:
        c+=1
        final_agg.append(a)

In [496]:
final_agg[0].keys()

[u'family',
 u'vgood',
 u'sep_nov',
 'cluster_id',
 'at',
 'country_code',
 u'mar_may',
 'visa_req',
 u'friend',
 u'poor',
 u'solo',
 u'business',
 u'couple',
 u'jun_aug',
 'allowed_stay',
 u'excellent',
 'dt',
 u'dec_feb',
 'country',
 u'average',
 u'terrible',
 '_id']

In [486]:

# put final_agg in mongo
import pymongo
client = pymongo.MongoClient()
db = client['tripadvisor']
collection2 = db['final_countries_data']
r=collection2.insert_many(final_agg)



In [450]:
c = {}
for x in range(17):
    tmp =[]
    ind= np.where(pred_cluster_ids_t ==x )
    #print len(ind[0])
    for i in ind[0]:
        tmp.append(Y_labels[i])
    c[x]= tmp

with open('countries_by_cluster.p', 'wb') as handle:
    pickle.dump(c, handle)

In [443]:
distr

{0: 14,
 1: 24,
 2: 9,
 3: 35,
 4: 10,
 5: 10,
 6: 3,
 7: 7,
 8: 13,
 9: 10,
 10: 9,
 11: 8,
 12: 6,
 13: 13,
 14: 11,
 15: 8,
 16: 13}

In [442]:
with open('cluster_distributions.p', 'wb') as handle:
    pickle.dump(distr, handle)

In [203]:
country_visas = pickle.load( open( "country_visas.p", "rb" ) )

In [225]:
def checkcode(code):
    for c in agg_countries:
        if 'country_code' in c:
            if c['country_code'] == code:
                return c
        

In [245]:
#final_agg2 = []
no_info = []
for ccode, country_visa in country_visas.items():
    country_dic = checkcode(ccode)
    if country_dic != None:
        country_dic['visa_req']= country_visa[0]
        country_dic['allowed_stay']= country_visa[1]
    else:
        no_info.append((ccode, country_visa))

In [246]:
no_info

[('COG', ['Visa required', '']),
 ('USA', ['Already from here!', '']),
 ('CIV', [u'eVisa', u''])]

In [217]:
country_visas['ZMB']

[u'Visa on arrival',
 u'90 days (exact period determined on arrival, later extendable free of charge)']

In [244]:
agg_countries[179]['country_code']= 'COD'

In [249]:
for i,a in enumerate(agg_countries):
    if a['country']=="Republic of Congo":
        print i

In [389]:
len(country_codes)

175